In [15]:
import pandas as pd
import datetime
import numpy as np
import pytz
import scipy
import matplotlib.pyplot as plt
from collections import OrderedDict
from numpy import linalg as LA
import time
from matplotlib import style
from scipy import signal
from scipy import fftpack
import pywt


style.use('dark_background')

# Accelerometer Data Feature Extraction

### 1) Reading John Doe's Accelerometer data 

In [16]:
df_acc = pd.read_csv("../../../data/accelerometer/835b51bd-ee31-49e8-a653-cb75a7e4c98e.csv", header=None)
df_acc.columns = ['_id1', '_id2', 'timestamp', 'device_id', 'double_x', 'double_y', 'double_z', 'accuracy', 'label']
df_acc.device_id='John Doe'
df_acc = df_acc.sort_values(by=['timestamp']).reset_index(drop=True)[['timestamp', 'device_id', 'double_x', 'double_y', 'double_z', 'accuracy', 'label']]
df_acc = df_acc[0:1000000]

In [17]:
def ecdfRep(data, components):
    #
    #   rep = ecdfRep(data, components)
    #
    #   Estimate ecdf-representation according to 
    #     Hammerla, Nils Y., et al. "On preserving statistical characteristics of 
    #     accelerometry data using their empirical cumulative distribution." 
    #     ISWC. ACM, 2013.
    #
    #   Input: 
    #       data        Nxd     Input data (rows = samples).
    #       components  int     Number of components to extract per axis.
    #
    #   Output:
    #       rep         Mx1     Data representation with M = d*components+d
    #                           elements.
    #
    #   Nils Hammerla '15
    #
        m = data.mean(0)
        data = np.sort(data, axis=0)
        data = data[np.int32(np.around(np.linspace(0,data.shape[0]-1,num=components))),:]
        data = data.flatten(1)
        return np.hstack((data, m))

In [18]:
"""
modes 
0 - time domain only
1 - time + frequency domain
2 - time + frequency + stats
3 - statistical methods only3
"""
def featurize_window(df_fw, feature_list, mode, window_size_in_minutes):
    local_dict = OrderedDict()
    
    
    if mode > 0 and mode < 3:
        if df_fw.index.size >= (30*window_size_in_minutes):
            df_fw.double_x = df_fw.double_x.replace({0:1e-08})
            df_fw.double_y = df_fw.double_y.replace({0:1e-08})
            df_fw.double_z = df_fw.double_z.replace({0:1e-08})
            f_x = scipy.interpolate.interp1d(df_fw.timestamp, df_fw.double_x)
            f_y = scipy.interpolate.interp1d(df_fw.timestamp, df_fw.double_y)
            f_z = scipy.interpolate.interp1d(df_fw.timestamp, df_fw.double_z)
            r = (np.sqrt(df_fw.double_x**2 + df_fw.double_y**2 + df_fw.double_z**2)).replace({0:1e-08})
            f_r = scipy.interpolate.interp1d(df_fw.timestamp, r)
            xnew = []
            step = (df_fw.timestamp.iloc[-1] - df_fw.timestamp.iloc[0]) /df_fw.index.size
            for ti in range(df_fw.timestamp.iloc[0], df_fw.timestamp.iloc[-1], int(step)):
                xnew.append(ti)
            
            f_fs = window_size_in_minutes * 60 / df_fw.index.size
            L = 512 # change it to 512
            local_dict.update({'skip_fft':True, 'fx': f_x(xnew), 'fy': f_y(xnew), 'fz': f_z(xnew), 'fr': f_r(xnew), 'fs': f_fs, 'L': L})
        else:
            local_dict.update({'skip_fft':False})
        if df_fw.index.size == 0:
            local_dict['skip_td'] = False
        else:
            local_dict['skip_td'] = True
            
    if mode == 0:
        local_dict['skip_fft'] = True
    if mode == 3:
        local_dict['skip_fft'] = True
        local_dict['skip_td'] = True

        
    feat_dict = OrderedDict()

    

    for feature in feature_list:
        if feature == 'int_desc':
            if local_dict['skip_td']:
                int_desc = np.sqrt((df_fw.double_x ** 2).describe() + (df_fw.double_y **2).describe() + (df_fw.double_z ** 2).describe())
                feat_dict.update({'int_mean': int_desc[1], 'int_std': int_desc[2], 
                                  'int_min': int_desc[3],'int_25': int_desc[4], 'int_50': int_desc[5],'int_75': int_desc[6]})
            else:
                feat_dict.update({'int_mean': np.nan, 'int_std': np.nan, 
                                  'int_min': np.nan,'int_25': np.nan, 'int_50': np.nan,'int_75': np.nan})
        elif feature == 'int_rms':
            if local_dict['skip_td']:
                int_rms = np.sqrt((df_fw.double_x**2).sum() + (df_fw.double_y**2).sum() + (df_fw.double_z**2).sum()) / np.sqrt(df_fw.index.size)
                feat_dict.update({'int_rms':int_rms})
            else:
                feat_dict.update({'int_rms': np.nan})
        elif feature == 'mag_desc':
            if local_dict['skip_td']:
                mag_desc = np.sqrt(df_fw.double_x**2 + df_fw.double_y**2 + df_fw.double_z**2).describe()
                feat_dict.update({'mag_mean': mag_desc[1], 'mag_std': mag_desc[2], 'mag_min': mag_desc[3],
                                  'mag_25': mag_desc[4], 'mag_50': mag_desc[5],'mag_75': mag_desc[6]})
            else:
                feat_dict.update({'mag_mean': np.nan, 'mag_std': np.nan, 'mag_min': np.nan,
                  'mag_25': np.nan, 'mag_50': np.nan,'mag_75': np.nan})
        elif feature == 'pear_coef':
            if local_dict['skip_td']:
                cov_matrix =  np.cov(np.stack((df_fw.double_x,df_fw.double_y, df_fw.double_z), axis=0))
                pear_coef_xy = cov_matrix[0,1] / (df_fw.double_x.std() * df_fw.double_y.std())
                pear_coef_yz = cov_matrix[1,2] / (df_fw.double_y.std() * df_fw.double_z.std())
                pear_coef_xz = cov_matrix[0,2] / (df_fw.double_x.std() * df_fw.double_z.std())
                feat_dict.update({'pear_coef_xy':pear_coef_xy, 'pear_coef_yz':pear_coef_yz,'pear_coef_xz':pear_coef_xz })
            else:
                feat_dict.update({'pear_coef_xy':np.nan, 'pear_coef_yz':np.nan,'pear_coef_xz':np.nan})
        elif feature == 'sma':
            if local_dict['skip_td']:
                sma = (np.abs(df_fw.double_x.to_numpy()).sum() + np.abs(df_fw.double_y.to_numpy()).sum() + np.abs(df_fw.double_z.to_numpy()).sum()) / df_fw.index.size
                feat_dict.update({'sma':sma})
            else:
                feat_dict.update({'sma':np.nan})
        elif feature == 'svm':
            if local_dict['skip_td']:
                svm = np.sqrt(df_fw.double_x**2 + df_fw.double_y**2 + df_fw.double_z**2).sum() / df_fw.index.size
                feat_dict.update({'svm':svm})
            else:
                feat_dict.update({'svm':np.nan})
        elif feature == 'fft':
            if local_dict['skip_fft']:
                L = local_dict['L']
                dfx = fftpack.fft(local_dict['fx'], 512)
                dfy = fftpack.fft(local_dict['fy'], 512)
                dfz = fftpack.fft(local_dict['fz'], 512)
                dfr = fftpack.fft(local_dict['fr'], 512)
                # DC component
                # Remove the L part!
                feat_dict.update({'fdc_x': np.mean(np.real(dfx)), 'fdc_y': np.mean(np.real(dfy)),
                                  'fdc_z':  np.mean(np.real(dfz)), 'fdc_r':  np.mean(np.real(dfr))})
                # Energy
                feat_dict.update({'feng_x': (np.sum(np.real(dfx)**2 + np.imag(dfx)**2)) / L, 'feng_y': (np.sum(np.real(dfy)**2 + np.imag(dfy)**2)) / L,
                                  'feng_z':  (np.sum(np.real(dfz)**2 + np.imag(dfz)**2)) / L, 'feng_r':  (np.sum(np.real(dfr)**2 + np.imag(dfr)**2)) / L})
                # Entropy
                ck_x = np.sqrt(np.real(dfx)**2  + np.imag(dfx)**2)
                cj_x = ck_x / np.sum(ck_x)
                e_x = np.sum(cj_x * np.log(cj_x))
                
                ck_y = np.sqrt(np.real(dfy)**2  + np.imag(dfy)**2)
                cj_y = ck_y / np.sum(ck_y)
                e_y = np.sum(cj_y * np.log(cj_y))
                
                ck_z = np.sqrt(np.real(dfz)**2  + np.imag(dfz)**2)
                cj_z = ck_z / np.sum(ck_z)
                e_z = np.sum(cj_z * np.log(cj_z))
                
                ck_r = np.sqrt(np.real(dfr)**2  + np.imag(dfr)**2)
                cj_r = ck_r / np.sum(ck_r)
                e_r = np.sum(cj_r * np.log(cj_r))
                
                feat_dict.update({'fent_x': e_x, 'fent_y':  e_y,'fent_z':  e_z, 'fent_r': e_r})
                
                # Correlation
                # Fix the length, should be FFT wndow size 512
            
                fcorr_xy = np.dot(np.real(dfx) / L, np.real(dfy) / L)
                fcorr_xz = np.dot(np.real(dfx) / L, np.real(dfz) / L)
                fcorr_yz = np.dot(np.real(dfy) / L, np.real(dfz) / L)
                
                feat_dict.update({'fcorr_xy': fcorr_xy,'fcorr_xz':  fcorr_xz, 'fcorr_yz': fcorr_yz})
                
            else:
                feat_dict.update({'fdc_x': np.nan, 'fdc_y':  np.nan,'fdc_z':  np.nan, 'fdc_r': np.nan})
                feat_dict.update({'feng_x':  np.nan, 'feng_y':  np.nan, 'feng_z':   np.nan, 'feng_r':   np.nan})
                feat_dict.update({'fent_x': np.nan, 'fent_y':  np.nan,'fent_z':  np.nan, 'fent_r': np.nan})
                feat_dict.update({'fcorr_xy': np.nan,'fcorr_xz':  np.nan, 'fcorr_yz': np.nan})
        elif feature == 'psd':
            if local_dict['skip_fft']:
                fs = local_dict['fs']
                psd_window = signal.get_window('boxcar', len(local_dict['fx'])) # do not pass this window
                freqs_x, pxx_denx = signal.periodogram(local_dict['fx'], window=psd_window, fs=fs)
                freqs_y, pxx_deny = signal.periodogram(local_dict['fy'], window=psd_window, fs=fs)
                freqs_z, pxx_denz = signal.periodogram(local_dict['fz'], window=psd_window, fs=fs)
                freqs_r, pxx_denr = signal.periodogram(local_dict['fr'], window=psd_window, fs=fs)
                feat_dict.update({'psd_mean_x': np.mean(pxx_denx), 'psd_mean_y': np.mean(pxx_deny),
                                  'psd_mean_z': np.mean(pxx_denz), 'psd_mean_r': np.mean(pxx_denr)})
                
                feat_dict.update({'psd_max_x': np.max(pxx_denx), 
                                  'psd_max_y': np.max(pxx_deny),
                                  'psd_max_z': np.max(pxx_denz), 
                                  'psd_max_r': np.max(pxx_denr)})
                
                
                freqs_05_3_x = np.argwhere((freqs_x >= 0.5) & (freqs_x <= 3))
                freqs_05_3_y = np.argwhere((freqs_y >= 0.5) & (freqs_y <= 3))
                freqs_05_3_z = np.argwhere((freqs_z >= 0.5) & (freqs_z <= 3))
                freqs_05_3_r = np.argwhere((freqs_r >= 0.5) & (freqs_r <= 3))
                
                
                # max b/w 0.3 - 3Hz 
                # 0.5 - 3 Hz if missing, maybe not 0.0
                feat_dict.update({'psd_max_x_05_3': np.max(pxx_denx[freqs_05_3_x]) if freqs_05_3_x.any() else 0.0, 
                  'psd_max_y_05_3': np.max(pxx_deny[freqs_05_3_y]) if freqs_05_3_y.any() else 0.0,
                  'psd_max_z_05_3': np.max(pxx_denz[freqs_05_3_z]) if freqs_05_3_z.any() else 0.0,
                  'psd_max_r_05_3': np.max(pxx_denr[freqs_05_3_r]) if freqs_05_3_r.any() else 0.0})
            else:
                feat_dict.update({'psd_mean_x': np.nan, 'psd_mean_y':np.nan,
                                  'psd_mean_z': np.nan, 'psd_mean_r': np.nan})
                feat_dict.update({'psd_max_x': np.nan, 
                                  'psd_max_y': np.nan,
                                  'psd_max_z': np.nan, 
                                  'psd_max_r': np.nan})
        elif feature == 'lmbs':
            if local_dict['skip_td']:
                lmb_f_05_3 = np.linspace(0.5, 3, 10000)
                lmb_psd_x = signal.lombscargle(df_fw.timestamp, df_fw.double_x, lmb_f_05_3, normalize=False)
                lmb_psd_y = signal.lombscargle(df_fw.timestamp, df_fw.double_y, lmb_f_05_3, normalize=False)
                lmb_psd_z = signal.lombscargle(df_fw.timestamp, df_fw.double_z, lmb_f_05_3, normalize=False)
                
                feat_dict.update({'lmb_psd_max_x_05_3': np.max(lmb_psd_x) if lmb_psd_x.any() else 0.0, 
                  'lmb_psd_max_y_05_3': np.max(lmb_psd_y) if lmb_psd_y.any() else 0.0,
                  'lmb_psd_max_z_05_3': np.max(lmb_psd_z) if lmb_psd_z.any() else 0.0})
            else:
                feat_dict.update({'lmb_psd_max_x_05_3': np.nan, 
                  'lmb_psd_max_y_05_3': np.nan,
                  'lmb_psd_max_z_05_3': np.nan})
                
                
    return feat_dict

In [19]:
# acc_features = ['int_desc', 'int_rms', 'mag_desc', 'pear_coef', 'sma', 'svm', 'ecdf_5', 'fft', 'psd', 'lmbs']
acc_features = ['int_desc', 'int_rms', 'mag_desc', 'pear_coef', 'sma', 'svm', 'ecdf_5', 'fft', 'psd']


def init_window_feature_df():
    return pd.DataFrame(columns= ['_window_id', 'start_time', 'end_time'])


def acc_windowizer(df_acc_sample, mode, window_size_in_minutes):
    window = 0
    window_start_time = df_acc_sample.timestamp.iloc[0]
    window_end_time = df_acc_sample.timestamp.iloc[-1]
    window_next_time = window_start_time + (datetime.timedelta(minutes=window_size_in_minutes).seconds * 10**3)
    window_id = 0
    window_start_index = 0
    df_acc_sample['window_id'] = -1
    df_feature_windows = init_window_feature_df()
    estimated_windows = (df_acc.timestamp.iloc[-1] - df_acc.timestamp.iloc[0]) / (window_size_in_minutes * 60000)
    print("Estimated no. of windows: ", estimated_windows)
    
    

    while window_next_time < window_end_time:
        window += 1
        print("window: ", window)
        print("Percentage: ", (window/estimated_windows) * 100, "%")
        window_indices = df_acc_sample.iloc[window_start_index:].timestamp < window_next_time
        df_acc_sample.window_id.iloc[window_start_index:][window_indices] = window_id
        feature_dict = featurize_window(df_acc_sample.iloc[window_start_index:][window_indices], acc_features, mode, window_size_in_minutes)
        feature_dict.update({'_window_id': window_id, 'start_time': window_start_time, 'end_time': window_next_time - 1, 'sample_count':window_indices[window_indices].index.size})
        df_feature_windows = df_feature_windows.append(feature_dict, ignore_index=True)
        window_start_time = window_next_time
        window_next_time = window_next_time + (datetime.timedelta(minutes=window_size_in_minutes).seconds * 10**3)
        window_id = window_id + 1
        window_start_index = window_indices[~window_indices].index[0]

    window_indices = df_acc_sample.iloc[window_start_index:].timestamp < window_next_time
    feature_dict = featurize_window(df_acc_sample.iloc[window_start_index:], acc_features, mode, window_size_in_minutes)
    df_acc_sample.window_id.iloc[window_start_index:][window_indices] = window_id
    feature_dict.update({'_window_id': window_id, 'start_time': window_start_time, 'end_time': window_next_time, 'sample_count':window_indices[window_indices].index.size})
    df_feature_windows = df_feature_windows.append(feature_dict, ignore_index=True)
    df_feature_windows.start_time = df_feature_windows.start_time.astype(np.int64)
    df_feature_windows.end_time = df_feature_windows.end_time.astype(np.int64)
    df_feature_windows.sample_count = df_feature_windows.sample_count.astype(np.int64)
    df_feature_windows._window_id = df_feature_windows._window_id.astype(np.int64)
    return df_feature_windows, df_acc_sample

In [20]:
df_feature_windows, df_acc_sample = acc_windowizer(df_acc, 2, 1)

Estimated no. of windows:  5405.43995
window:  1
Percentage:  0.018499881771880566 %
window:  2
Percentage:  0.03699976354376113 %
window:  3
Percentage:  0.0554996453156417 %
window:  4
Percentage:  0.07399952708752226 %
window:  5
Percentage:  0.09249940885940283 %
window:  6
Percentage:  0.1109992906312834 %
window:  7
Percentage:  0.12949917240316397 %
window:  8
Percentage:  0.14799905417504453 %
window:  9
Percentage:  0.1664989359469251 %
window:  10
Percentage:  0.18499881771880566 %
window:  11
Percentage:  0.20349869949068622 %
window:  12
Percentage:  0.2219985812625668 %


/usr/local/lib/python3.7/site-packages/scipy/interpolate/interpolate.py:610: RuntimeWarning: divide by zero encountered in true_divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
/usr/local/lib/python3.7/site-packages/scipy/interpolate/interpolate.py:613: RuntimeWarning: invalid value encountered in multiply
  y_new = slope*(x_new - x_lo)[:, None] + y_lo
/usr/local/lib/python3.7/site-packages/numpy/core/fromnumeric.py:83: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


window:  13
Percentage:  0.24049846303444733 %
window:  14
Percentage:  0.25899834480632794 %
window:  15
Percentage:  0.2774982265782085 %
window:  16
Percentage:  0.29599810835008905 %
window:  17
Percentage:  0.3144979901219696 %
window:  18
Percentage:  0.3329978718938502 %
window:  19
Percentage:  0.3514977536657307 %
window:  20
Percentage:  0.3699976354376113 %
window:  21
Percentage:  0.3884975172094919 %
window:  22
Percentage:  0.40699739898137244 %
window:  23
Percentage:  0.42549728075325305 %
window:  24
Percentage:  0.4439971625251336 %
window:  25
Percentage:  0.46249704429701416 %
window:  26
Percentage:  0.48099692606889466 %
window:  27
Percentage:  0.4994968078407753 %
window:  28
Percentage:  0.5179966896126559 %
window:  29
Percentage:  0.5364965713845364 %
window:  30
Percentage:  0.554996453156417 %
window:  31
Percentage:  0.5734963349282975 %
window:  32
Percentage:  0.5919962167001781 %
window:  33
Percentage:  0.6104960984720587 %
window:  34
Percentage:  0.6

window:  191
Percentage:  3.533477418429188 %
window:  192
Percentage:  3.551977300201069 %
window:  193
Percentage:  3.5704771819729495 %
window:  194
Percentage:  3.5889770637448297 %
window:  195
Percentage:  3.6074769455167104 %
window:  196
Percentage:  3.6259768272885915 %
window:  197
Percentage:  3.6444767090604713 %
window:  198
Percentage:  3.6629765908323524 %
window:  199
Percentage:  3.681476472604233 %
window:  200
Percentage:  3.6999763543761133 %
window:  201
Percentage:  3.718476236147994 %
window:  202
Percentage:  3.7369761179198746 %
window:  203
Percentage:  3.755475999691755 %
window:  204
Percentage:  3.7739758814636355 %
window:  205
Percentage:  3.7924757632355166 %
window:  206
Percentage:  3.8109756450073964 %
window:  207
Percentage:  3.8294755267792775 %
window:  208
Percentage:  3.8479754085511573 %
window:  209
Percentage:  3.8664752903230384 %
window:  210
Percentage:  3.884975172094919 %
window:  211
Percentage:  3.9034750538667993 %
window:  212
Percen

window:  371
Percentage:  6.863456137367691 %
window:  372
Percentage:  6.881956019139571 %
window:  373
Percentage:  6.900455900911451 %
window:  374
Percentage:  6.918955782683332 %
window:  375
Percentage:  6.937455664455213 %
window:  376
Percentage:  6.955955546227092 %
window:  377
Percentage:  6.974455427998974 %
window:  378
Percentage:  6.9929553097708546 %
window:  379
Percentage:  7.011455191542734 %
window:  380
Percentage:  7.029955073314616 %
window:  381
Percentage:  7.048454955086496 %
window:  382
Percentage:  7.066954836858376 %
window:  383
Percentage:  7.085454718630258 %
window:  384
Percentage:  7.103954600402138 %
window:  385
Percentage:  7.122454482174017 %
window:  386
Percentage:  7.140954363945899 %
window:  387
Percentage:  7.15945424571778 %
window:  388
Percentage:  7.177954127489659 %
window:  389
Percentage:  7.196454009261541 %
window:  390
Percentage:  7.214953891033421 %
window:  391
Percentage:  7.233453772805301 %
window:  392
Percentage:  7.251953

...

window:  4815
Percentage:  89.07693073160493 %
window:  4816
Percentage:  89.09543061337682 %
window:  4817
Percentage:  89.11393049514868 %
window:  4818
Percentage:  89.13243037692057 %
window:  4819
Percentage:  89.15093025869245 %
window:  4820
Percentage:  89.16943014046433 %
window:  4821
Percentage:  89.1879300222362 %
window:  4822
Percentage:  89.20642990400809 %
window:  4823
Percentage:  89.22492978577998 %
window:  4824
Percentage:  89.24342966755185 %
window:  4825
Percentage:  89.26192954932374 %
window:  4826
Percentage:  89.28042943109561 %
window:  4827
Percentage:  89.29892931286749 %
window:  4828
Percentage:  89.31742919463937 %
window:  4829
Percentage:  89.33592907641126 %
window:  4830
Percentage:  89.35442895818314 %
window:  4831
Percentage:  89.37292883995501 %
window:  4832
Percentage:  89.3914287217269 %
window:  4833
Percentage:  89.40992860349878 %
window:  4834
Percentage:  89.42842848527066 %
window:  4835
Percentage:  89.44692836704255 %
window:  4836
P

window:  4991
Percentage:  92.3329099234559 %
window:  4992
Percentage:  92.35140980522779 %
window:  4993
Percentage:  92.36990968699968 %
window:  4994
Percentage:  92.38840956877155 %
window:  4995
Percentage:  92.40690945054342 %
window:  4996
Percentage:  92.42540933231531 %
window:  4997
Percentage:  92.4439092140872 %
window:  4998
Percentage:  92.46240909585907 %
window:  4999
Percentage:  92.48090897763095 %
window:  5000
Percentage:  92.49940885940283 %
window:  5001
Percentage:  92.51790874117472 %
window:  5002
Percentage:  92.5364086229466 %
window:  5003
Percentage:  92.55490850471847 %
window:  5004
Percentage:  92.57340838649036 %
window:  5005
Percentage:  92.59190826826223 %
window:  5006
Percentage:  92.61040815003412 %
window:  5007
Percentage:  92.628908031806 %
window:  5008
Percentage:  92.64740791357788 %
window:  5009
Percentage:  92.66590779534975 %
window:  5010
Percentage:  92.68440767712164 %
window:  5011
Percentage:  92.70290755889351 %
window:  5012
Perc

window:  5168
Percentage:  95.60738899707877 %
window:  5169
Percentage:  95.62588887885065 %
window:  5170
Percentage:  95.64438876062253 %
window:  5171
Percentage:  95.6628886423944 %
window:  5172
Percentage:  95.6813885241663 %
window:  5173
Percentage:  95.69988840593817 %
window:  5174
Percentage:  95.71838828771006 %
window:  5175
Percentage:  95.73688816948193 %
window:  5176
Percentage:  95.7553880512538 %
window:  5177
Percentage:  95.77388793302569 %
window:  5178
Percentage:  95.79238781479758 %
window:  5179
Percentage:  95.81088769656945 %
window:  5180
Percentage:  95.82938757834133 %
window:  5181
Percentage:  95.84788746011321 %
window:  5182
Percentage:  95.8663873418851 %
window:  5183
Percentage:  95.88488722365697 %
window:  5184
Percentage:  95.90338710542886 %
window:  5185
Percentage:  95.92188698720074 %
window:  5186
Percentage:  95.94038686897261 %
window:  5187
Percentage:  95.9588867507445 %
window:  5188
Percentage:  95.97738663251639 %
window:  5189
Perc

window:  5349
Percentage:  98.95586759778915 %
window:  5350
Percentage:  98.97436747956104 %
window:  5351
Percentage:  98.99286736133291 %
window:  5352
Percentage:  99.0113672431048 %
window:  5353
Percentage:  99.02986712487667 %
window:  5354
Percentage:  99.04836700664855 %
window:  5355
Percentage:  99.06686688842044 %
window:  5356
Percentage:  99.08536677019232 %
window:  5357
Percentage:  99.1038666519642 %
window:  5358
Percentage:  99.12236653373607 %
window:  5359
Percentage:  99.14086641550796 %
window:  5360
Percentage:  99.15936629727983 %
window:  5361
Percentage:  99.17786617905172 %
window:  5362
Percentage:  99.19636606082359 %
window:  5363
Percentage:  99.21486594259548 %
window:  5364
Percentage:  99.23336582436735 %
window:  5365
Percentage:  99.25186570613924 %
window:  5366
Percentage:  99.27036558791113 %
window:  5367
Percentage:  99.288865469683 %
window:  5368
Percentage:  99.30736535145488 %
window:  5369
Percentage:  99.32586523322676 %
window:  5370
Per

/usr/local/lib/python3.7/site-packages/pandas/core/generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [23]:
df_feature_windows[df_feature_windows.psd_max_r_05_3 > 0.0]

,_window_id,start_time,end_time,fcorr_xy,fcorr_xz,fcorr_yz,fdc_r,fdc_x,fdc_y,fdc_z,...,psd_max_y_05_3,psd_max_z,psd_max_z_05_3,psd_mean_r,psd_mean_x,psd_mean_y,psd_mean_z,sample_count,sma,svm
4218,4218,1550860119334,1550860179333,-0.010487,-0.007274,0.019060,1.162081,0.173584,-0.509903,-1.029709,...,0.006187,0.447557,0.010505,0.005495,0.020992,0.032226,0.038160,47,1.590902,1.006618
4568,4568,1550881119334,1550881179333,0.000138,0.001678,0.002945,1.006940,-0.060303,-0.076965,-1.002182,...,0.000024,0.000095,0.000064,0.000032,0.000099,0.000084,0.000030,34,1.130731,1.004094
4611,4611,1550883699334,1550883759333,-0.003514,-0.003222,0.019375,1.022129,0.175461,-0.743362,-0.679245,...,0.002385,0.048589,0.003476,0.000677,0.013807,0.004697,0.004462,39,1.537033,1.003826


In [43]:
df_acc.replace({0:1e-08})

,timestamp,device_id,double_x,double_y,double_z,accuracy,label,window_id
0,1.550607e+12,John Doe,0.003357,-0.003662,-1.001831,3.0,NaN,1.000000e-08
1,1.550607e+12,John Doe,0.001236,-0.005814,-1.003357,3.0,NaN,1.000000e-08
2,1.550607e+12,John Doe,0.003067,-0.004547,-1.004333,3.0,NaN,1.000000e-08
3,1.550607e+12,John Doe,0.002472,-0.005539,-1.001297,3.0,NaN,1.000000e-08
4,1.550607e+12,John Doe,0.001236,-0.004761,-1.002701,3.0,NaN,1.000000e-08
...,...,...,...,...,...,...,...,...
99995,1.550633e+12,John Doe,-0.429092,-0.067642,-0.901794,3.0,NaN,4.250000e+02
99996,1.550633e+12,John Doe,-0.426941,-0.067368,-0.903107,3.0,NaN,4.250000e+02
99997,1.550633e+12,John Doe,-0.425339,-0.066910,-0.906387,3.0,NaN,4.250000e+02
99998,1.550633e+12,John Doe,-0.426346,-0.068665,-0.904526,3.0,NaN,4.250000e+02


In [44]:
df_acc

,timestamp,device_id,double_x,double_y,double_z,accuracy,label,window_id
0,1550607039334,John Doe,0.003357,-0.003662,-1.001831,3,NaN,0
1,1550607040049,John Doe,0.001236,-0.005814,-1.003357,3,NaN,0
2,1550607040050,John Doe,0.003067,-0.004547,-1.004333,3,NaN,0
3,1550607040056,John Doe,0.002472,-0.005539,-1.001297,3,NaN,0
4,1550607040128,John Doe,0.001236,-0.004761,-1.002701,3,NaN,0
...,...,...,...,...,...,...,...,...
99995,1550632568796,John Doe,-0.429092,-0.067642,-0.901794,3,NaN,425
99996,1550632568994,John Doe,-0.426941,-0.067368,-0.903107,3,NaN,425
99997,1550632569193,John Doe,-0.425339,-0.066910,-0.906387,3,NaN,425
99998,1550632569391,John Doe,-0.426346,-0.068665,-0.904526,3,NaN,425


In [36]:
df_acc[df_acc.double_x == 0]

,timestamp,device_id,double_x,double_y,double_z,accuracy,label,window_id
1386,1550607314207,John Doe,0.0,-0.002228,-1.003815,3,NaN,4
1840,1550607404374,John Doe,0.0,0.007553,-1.003220,3,NaN,6
2142,1550607464415,John Doe,0.0,-0.010956,-1.008286,3,NaN,7
44398,1550621393380,John Doe,0.0,-0.011353,0.994781,3,NaN,239
44827,1550621478604,John Doe,0.0,-0.012650,0.994339,3,NaN,240
...,...,...,...,...,...,...,...,...
64839,1550625455085,John Doe,0.0,-0.039612,-1.001740,3,NaN,306
65120,1550625510760,John Doe,0.0,-0.039948,-1.001358,3,NaN,307
68774,1550626236287,John Doe,0.0,-0.039566,-1.000595,3,NaN,319
71169,1550626766450,John Doe,0.0,-0.777435,-0.633713,3,NaN,328


In [33]:
df_feature_windows

,_window_id,start_time,end_time,fcorr_xy,fcorr_xz,fcorr_yz,fdc_r,fdc_x,fdc_y,fdc_z,...,psd_max_y_05_3,psd_max_z,psd_max_z_05_3,psd_mean_r,psd_mean_x,psd_mean_y,psd_mean_z,sample_count,sma,svm
0,0,1550607039334,1550607099333,-0.000009,-0.001276,0.001991,1.001843,0.003357,-0.003662,-1.001831,...,0.0,0.004785,0.0,0.001410,0.000147,0.000099,0.001385,303,1.012496,1.003649
1,1,1550607099334,1550607159333,-0.000007,-0.001772,0.001922,1.007780,0.010849,-0.005646,-1.007706,...,0.0,0.002720,0.0,0.000608,0.000185,0.000459,0.000602,303,1.014064,1.003088
2,2,1550607159334,1550607219333,-0.000009,-0.001088,0.002145,1.002321,0.006104,-0.009369,-1.002258,...,0.0,0.002463,0.0,0.000555,0.000153,0.000274,0.000541,303,1.012675,1.003294
3,3,1550607219334,1550607279333,-0.000132,-0.003956,0.002267,1.002109,0.002457,-0.005493,-1.002090,...,0.0,0.014926,0.0,0.001354,0.021394,0.002198,0.003341,302,1.019092,1.003243
4,4,1550607279334,1550607339333,0.000318,-0.006994,-0.000188,0.917915,0.592087,0.036926,-0.700455,...,0.0,0.129441,0.0,0.020595,0.100604,0.004103,0.024326,301,1.028899,1.001794
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421,421,1550632299334,1550632359333,0.004550,-0.077058,-0.004370,0.991277,-0.283813,-0.110107,0.943375,...,0.0,29.953732,0.0,0.072112,0.334396,0.579086,1.303667,303,1.364199,0.998353
422,422,1550632359334,1550632419333,-0.002518,-0.009684,0.151409,1.075836,0.047562,-0.674133,-0.837082,...,0.0,23.982404,0.0,0.011128,0.034919,0.617888,0.226967,302,1.404371,1.006385
423,423,1550632419334,1550632479333,0.000037,0.016526,-0.018062,1.004284,0.062668,0.040451,-1.001511,...,0.0,0.472684,0.0,0.007880,0.217300,0.048755,0.028224,301,1.149429,1.002986
424,424,1550632479334,1550632539333,0.008123,0.146608,0.017986,1.003222,-0.399185,-0.064926,-0.918091,...,0.0,0.001140,0.0,0.000093,0.000230,0.000290,0.000106,303,1.373938,1.001329


In [32]:
df_acc

,timestamp,device_id,double_x,double_y,double_z,accuracy,label,window_id
0,1550607039334,John Doe,0.003357,-0.003662,-1.001831,3,NaN,0
1,1550607040049,John Doe,0.001236,-0.005814,-1.003357,3,NaN,0
2,1550607040050,John Doe,0.003067,-0.004547,-1.004333,3,NaN,0
3,1550607040056,John Doe,0.002472,-0.005539,-1.001297,3,NaN,0
4,1550607040128,John Doe,0.001236,-0.004761,-1.002701,3,NaN,0
...,...,...,...,...,...,...,...,...
99995,1550632568796,John Doe,-0.429092,-0.067642,-0.901794,3,NaN,425
99996,1550632568994,John Doe,-0.426941,-0.067368,-0.903107,3,NaN,425
99997,1550632569193,John Doe,-0.425339,-0.066910,-0.906387,3,NaN,425
99998,1550632569391,John Doe,-0.426346,-0.068665,-0.904526,3,NaN,425


In [ ]:
df_feature_windows.sample_count

In [ ]:
df_feature_windows.sample_count.describe()

In [ ]:
df_feature_windows.columns.tolist()

In [ ]:
cov_matrix = np.cov(np.stack((df_acc[0:100].double_x,  df_acc[0:100].double_y, df_acc[0:100].double_z), axis=0))

In [ ]:
cov_matrix / (df_acc[0:100].double_y.std()*df_acc[0:100].double_z.std())

In [ ]:
A = [12,16,13,18,19,12,18,19,12,14]
B = [73,67,74,63,73,84,60,62,76,71]


In [ ]:
np.std(A)

In [ ]:
np.std(B)

In [ ]:
np.cov(np.stack((A, B), axis=0)) / (np.std(A, ddof=1) * np.std(B, ddof=1))

In [ ]:
cov_matrix

In [ ]:
np.correlate(df_acc[0:100].double_x,df_acc[0:100].double_y )

In [ ]:
np.abs(df_acc[0:100].double_x.to_numpy()).sum() + 

In [ ]:
def stft(x, NFFT=256, fs=2.0, noverlap=128, windowfunc=hann, zeropadding="sandwich"):
    """
    Calculate the short-time fourier transform of an input signal x

    Parameters
    ----------
    x : 1-D signal array
    Preferrably a numpy array
    NFFT : int
    Desired frequency resolution in bins (default 256)
    fs : float
    Sampling rate in Hz (default 2.0)
    noverlap : int
    Number of samples to overlap analysis windows (default 128)
    windowfunc : function
    Window function to apply before fft analysis (default `scipy.signal.hann`)
    zeropadding : string
    What kind of zero-padding to apply to `x` before analysis. One of:
    * "after": `pad_amnt` zeros are added to the end of `x`
    * "sandwich": `NFFT/2` zeros are added to beginning of `x`, followed by
    `NFFT/2 + pad_amnt` zeros added to the end of `x`
    `pad_amnt` is the number of samples required to pad `x` to the nearest
    integer multiple of `NFFT`. Default padding strategy is "sandwich", which
    yields nice properties when lining up time indices
    """

    if noverlap >= NFFT:
        raise ValueError("`noverlap` (%d) must be less than or equal to `NFFT` (%d)" % (noverlap, NFFT))
    step = (NFFT - noverlap)
    pad_amnt = step - len(x) % step

    nlen = len(x) // step
    NFFT_2 = NFFT // 2
    P = empty((NFFT_2, nlen))
    window = windowfunc(NFFT)

    if zeropadding == "sandwich":
        x = hstack((zeros(NFFT_2), x, zeros(NFFT_2 + pad_amnt)))
    elif zeropadding == "after":
        x = hstack((x, zeros(pad_amnt)))
    else:
        raise ValueError("Unrecognized `zeropadding` value: " + zeropadding)

    for idx in range(nlen):
        x_slice = x[idx step:idx step + NFFT]
        P[:, idx] = abs(fft(x_slice * window)[:NFFT_2])
    return P

In [ ]:
time.mktime(df_acc.timestamp.iloc[0]/1000)


In [ ]:
time_window = 60

In [ ]:
(df_acc.timestamp.iloc[-1] - df_acc.timestamp.iloc[0]) / (time_window * 60000)

In [ ]:
cA, cD = pywt.dwt(df_acc[0:30].double_x, 'haar')

In [ ]:
pd.DataFrame(cD).plot()

In [ ]:
df_acc[0:30].double_x.plot()

In [ ]:
def ecdfRep(data, components):
    #
    #   rep = ecdfRep(data, components)
    #
    #   Estimate ecdf-representation according to 
    #     Hammerla, Nils Y., et al. "On preserving statistical characteristics of 
    #     accelerometry data using their empirical cumulative distribution." 
    #     ISWC. ACM, 2013.
    #
    #   Input: 
    #       data        Nxd     Input data (rows = samples).
    #       components  int     Number of components to extract per axis.
    #
    #   Output:
    #       rep         Mx1     Data representation with M = d*components+d
    #                           elements.
    #
    #   Nils Hammerla '15
    #
    m = data.mean(0)
    data = np.sort(data, axis=0)
    data = data[np.int32(np.around(np.linspace(0,data.shape[0]-1,num=components))),:]
    data = data.flatten(1)
    return np.hstack((data, m))




In [ ]:
a = np.random.randn(100,3)
e1 = ecdfRep(a,10)
print(e1)


In [ ]:
df_feature_windows.sample_count.describe()

In [ ]:
pd.DataFrame(a)[0].plot()

In [ ]:
plt.plot(df_acc_sample[df_acc_sample.window_id==0].timestamp, df_acc_sample[df_acc_sample.window_id==0].double_x)

In [ ]:
f = scipy.interpolate.interp1d(df_acc_sample[df_acc_sample.window_id==0].timestamp, df_acc_sample[df_acc_sample.window_id==0].double_x)

In [ ]:
df_acc_sample[df_acc_sample.window_id==0].double_x**2 + df_acc_sample[df_acc_sample.window_id==0].double_y**2+ df_acc_sample[df_acc_sample.window_id==0].double_z**2

In [ ]:
plt.plot(df_acc_sample[df_acc_sample.window_id==0].timestamp, f(df_acc_sample[df_acc_sample.window_id==0].timestamp))

In [ ]:
xnew = []
step = (df_acc_sample[df_acc_sample.window_id==0].timestamp.iloc[-1] - df_acc_sample[df_acc_sample.window_id==0].timestamp.iloc[0]) / df_acc_sample[df_acc_sample.window_id==0].index.size
for i in range(df_acc_sample[df_acc_sample.window_id==0].timestamp.iloc[0], df_acc_sample[df_acc_sample.window_id==0].timestamp.iloc[-1], int(step)):
    xnew.append(i)
plt.plot(xnew, f(xnew))

In [ ]:
ecdfRep(np.array(df_acc_sample[df_acc_sample.window_id==0].double_x.tolist()).reshape([482,1]), 4)

In [ ]:
x_axis = np.array(df_acc_sample[df_acc_sample.window_id==0].double_x.tolist()).reshape([482,1])
x_axis = np.array(df_acc_sample[df_acc_sample.window_id==0].double_y.tolist()).reshape([482,1])
z_axis = np.array(df_acc_sample[df_acc_sample.window_id==0].double_z.tolist()).reshape([482,1])
accel_data = np.concatenate([x_axis,x_axis,z_axis], axis=1)

In [ ]:
a = np.random.randn(100,3)
e1 = ecdfRep(accel_data,4)
print(e1)


In [ ]:
sgnal = f(xnew)
L = len(sgnal)
freqs, Pxx_den = signal.periodogram(sgnal, window=signal.get_window('boxcar', len(sgnal)), fs=8.1)
plt.semilogy(freqs, Pxx_den)
plt.ylim([1e-5, 1e2])
plt.xlabel('frequency [Hz]')
plt.ylabel('PSD [V**2/Hz]')
plt.show()

In [ ]:
int_fs = np.argwhere((freqs >= 0.5) & (freqs <= 3))
freqs_int = freqs[int_fs]
pdd = Pxx_den[int_fs]
np.argmax(pdd)
freqs_int[14]

In [ ]:
lmbs_f = np.linspace(0.5, 3, 10000)
pgram = signal.lombscargle(df_acc_sample[df_acc_sample.window_id==0].timestamp, df_acc_sample[df_acc_sample.window_id==0].double_x, lmbs_f, normalize=True)
plt.plot(lmbs_f, pgram)
lmbs_f[np.argmax(pgram)]

In [ ]:
Pxx_den = Pxx_den

In [ ]:
np.mean(Pxx_den) # PSD 

In [ ]:
Y = fftpack.fft(sgnal, 512)
np.dot(np.real(Y), np.real(Y))

In [ ]:
(np.sum(np.real(Y)**2 + np.imag(Y)**2)) / L

In [ ]:
np.sqrt(np.real(Y)**2  + np.imag(Y)**2) / np.sum(np.sqrt(np.real(Y)**2  + np.imag(Y)**2))

In [ ]:
LA.norm(np.mean(Y[1:]))

In [ ]:
f, Pxx_spec = signal.periodogram(sgnal, 8.1, 'flattop', scaling='spectrum')
plt.figure()
plt.semilogy(f, np.sqrt(Pxx_spec))
plt.xlabel('frequency [Hz]')
plt.ylabel('Linear spectrum [V RMS]')
plt.show()

In [ ]:
np.mean(Pxx_spec)

In [ ]:
Amplitude = np.abs(sig_fft)
psd = Amplitude**2
Angle = np.angle(sig_fft)

In [ ]:
sample_freq = fftpack.fftfreq(f(xnew).size, d=0.05)
sample_freq

In [ ]:
df_feature_windows.index.size

In [ ]:
ld = OrderedDict()
ld.update({'min':True})

In [ ]:
if ld['min']:
    print("yes")

In [ ]:
xx= ['int_desc', 'int_rms', 'mag_desc', 'pear_coef', 'sma', 'svm', 'ecdf_5', 'dc_comp', 'psd', 'psd_welch', 'psd_ls']

In [ ]:
for x in xx:
    print(x)

In [ ]:
import matplotlib.pyplot as plt
A = 2.
w = 1.
phi = 0.5 * np.pi
nin = 1000
nout = 100000
frac_points = 0.9 # Fraction of points to select
r = np.random.rand(nin)
x = np.linspace(0.01, 10*np.pi, nin)
x = x[r >= frac_points]
x

In [ ]:
y = A * np.sin(w*x+phi)

In [ ]:
plt.plot(x,y)

In [ ]:
f = np.linspace(0.01, 10, nout)
f

In [ ]:
import scipy.signal as signal
pgram = signal.lombscargle(x, y, f, normalize=False)

In [ ]:
plt.subplot(2, 1, 1)
plt.plot(x, y, 'b+')

In [ ]:
plt.plot(f, pgram)


In [ ]:
np.max(pgram)
f[np.argmax(pgram)]

In [ ]:
x